<ul class="breadcrumb">
  <li><a href="1_Conventional_Linear_Mixed_Model.ipynb">Bayesian Linear Mixed Models (Conventional)</a></li>
  <li><a href="2_Linear_Additive_Genetic_Model.ipynb">Bayesian Linear Additive Genetic Model</a></li> 
  <li><a href="3_Genomic_Linear_Mixed_Model.ipynb">Bayesian Linear Mixed Models (Genomic Data)</a></li>
</ul>

<div class="span5 alert alert-success">
 <font size="5" face="Georgia">Single-step Bayesian Regression (Incomplete Genomic Data)</font> 
</div>

<button type="button" class="btn btn-lg btn-primary">Step 1: Load Packages</button> 

In [2]:
using JWAS,JWAS.Datasets,DataFrames,CSV

<button type="button" class="btn btn-lg btn-primary">Step 2: Read data</button> 

In [3]:
phenofile  = Datasets.dataset("example","phenotypes_ssbr.txt")
pedfile    = Datasets.dataset("example","pedigree.txt")
genofile   = Datasets.dataset("example","genotypes.txt")

phenotypes = CSV.read(phenofile,delim = ',',header=true)
pedigree   = get_pedigree(pedfile,separator=",",header=true);

coding pedigree... 100%|████████████████████████████████| Time: 0:00:00

Finished!



calculating inbreeding... 100%|█████████████████████████| Time: 0:00:00


In [4]:
head(phenotypes)

,ID,y1,y2,y3,x1,x2,x3,dam
1,a1,-0.06,3.58,-1.18,0.9,2,m,0
2,a2,-0.6,4.9,0.88,0.3,1,f,0
3,a3,-2.07,3.19,0.73,0.7,2,f,0
4,a4,-2.63,6.97,-0.83,0.6,1,m,a2
5,a5,2.31,3.5,-1.52,0.4,2,m,a2
6,a6,0.93,4.87,-0.01,5.0,2,f,a3


<div class="span5 alert alert-success">
 <font size="5" face="Georgia">Single-trait Single-step Bayesian Regression (Incomplete Genomic Data)</font> 
</div>

<button type="button" class="btn btn-lg btn-primary">Step 3: Build Model Equations</button> 

In [5]:
model_equation1  ="y1 = intercept + x1*x3 + x2 + x3 + ID + dam";

In [6]:
R      = 1.0
model1 = build_model(model_equation1,R);

<button type="button" class="btn btn-lg btn-primary">Step 4: Set Factors or Covariates</button> 

In [7]:
set_covariate(model1,"x1");

<button type="button" class="btn btn-lg btn-primary">Step 5: Set Random or Fixed Effects</button> 

In [8]:
G1 = 1.0
G2 = eye(2)
set_random(model1,"x2",G1);
set_random(model1,"ID dam",pedigree,G2);

<button type="button" class="btn btn-lg btn-primary">Step 6: Use Genomic Information</button> 

In [9]:
G3 =1.0
add_genotypes(model1,genofile,G3,separator=',');

5 markers on 7 individuals were added.


In [10]:
outputEBV(model1,["a1","a2","a3"]);

LoadError: [91mUndefVarError: outputEBV not defined[39m

<button type="button" class="btn btn-lg btn-primary">Step 7: Run Analysis</button> 

In [11]:
outputMCMCsamples(model1,"x2")
out1=runMCMC(model1,phenotypes,methods="RR-BLUP",single_step_analysis=true,pedigree=pedigree,chain_length=5000,output_samples_frequency=100);


The prior for marker effects variance is calculated from 
the genetic variance and π. The prior for the marker effects variance 
is: 0.492462



A Linear Mixed Model was build using model equations:

y1 = intercept + x1*x3 + x2 + x3 + ID + dam

Model Information:

Term            C/F          F/R            nLevels
intercept       factor       fixed                1
x1*x3           interaction  fixed                2
x2              factor       random               2
x3              factor       fixed                2
ID              factor       random              12
dam             factor       random              12
ϵ               factor       random               5
J               covariate    fixed                1

MCMC Information:

methods                                     RR-BLUP
chain_length                                   5000
burnin                                            0
estimatePi                                    false
starting_value                        

running MCMC for RR-BLUP...100%|████████████████████████| Time: 0:00:01


<button type="button" class="btn btn-lg btn-primary">Check Results</button> 

In [12]:
keys(out1)

Base.KeyIterator for a Dict{Any,Any} with 6 entries. Keys:
  "Posterior mean of polygenic effects covariance matrix"
  "EBV_y1"
  "Posterior mean of marker effects"
  "Posterior mean of residual variance"
  "Posterior mean of marker effects variance"
  "Posterior mean of location parameters"

In [13]:
out1["EBV_y1"]

8×2 Array{Any,2}:
 "a1"   -4.69148
 "a2"   -2.48478
 "a3"   -5.3797 
 "a4"   -5.49761
 "a5"   -4.02786
 "a6"   -4.86453
 "a10"  -5.25845
 "a12"  -4.51325

In [14]:
out1["Posterior mean of marker effects"]

5×2 Array{Any,2}:
 "m1"  -0.0327138
 "m2"  -0.125942 
 "m3"   0.274938 
 "m4"  -0.216556 
 "m5"  -0.0533177

In [15]:
out1["Posterior mean of location parameters"]

,Trait,Effect,Level,Estimate
1,1,intercept,intercept,-47.0364
2,1,x1*x3,x1 * m,-2.8794
3,1,x1*x3,x1 * f,0.717336
4,1,x2,2,0.167268
5,1,x2,1,-0.134571
6,1,x3,m,53.0247
7,1,x3,f,49.1598
8,1,ID,a12,0.240182
9,1,ID,a10,-0.319762
10,1,ID,a11,-0.0403208


<div class="span5 alert alert-success">
 <font size="5" face="Georgia">Multi-trait Single-step Bayesian Regression (Incomplete Genomic Data)</font> 
</div>

<button type="button" class="btn btn-lg btn-primary">Step 3: Build Model Equations</button> 

In [16]:
model_equation2 ="y1 = intercept + x1 + x3 + ID + dam
                  y2 = intercept + x1 + x2 + x3 + ID
                  y3 = intercept + x1 + x1*x3 + x2 + ID";

In [17]:
R      = eye(3)
model2 = build_model(model_equation2,R);

<button type="button" class="btn btn-lg btn-primary">Step 4: Set Factors or Covariates</button> 

In [18]:
set_covariate(model2,"x1");

<button type="button" class="btn btn-lg btn-primary">Step 5: Set Random or Fixed Effects</button> 

In [19]:
G1 = eye(2)
G2 = eye(4)
set_random(model2,"x2",G1);
set_random(model2,"ID dam",pedigree,G2);

INFO: x2 is not found in model equation 1.
INFO: dam is not found in model equation 2.
INFO: dam is not found in model equation 3.


<button type="button" class="btn btn-lg btn-primary">Step 6: Use Genomic Information</button> 

In [20]:
G3 = eye(3)
add_genotypes(model2,genofile,G3,separator=',');

5 markers on 7 individuals were added.


<button type="button" class="btn btn-lg btn-primary">Step 7: Run Analysis</button> 

In [21]:
out2=runMCMC(model2,phenotypes,methods="RR-BLUP",single_step_analysis=true,pedigree=pedigree,chain_length=5000,output_samples_frequency=100);

INFO: Pi (Π) is not provided.
INFO: Pi (Π) is generated assuming all markers have effects on all traits.



The prior for marker effects covariance matrix is calculated from 
genetic covariance matrix and Π. The prior for the marker effects 
covariance matrix is: 

 0.492462  0.0       0.0     
 0.0       0.492462  0.0     
 0.0       0.0       0.492462


A Linear Mixed Model was build using model equations:

y1 = intercept + x1 + x3 + ID + dam
y2 = intercept + x1 + x2 + x3 + ID
y3 = intercept + x1 + x1*x3 + x2 + ID

Model Information:

Term            C/F          F/R            nLevels
intercept       factor       fixed                1
x1              covariate    fixed                1
x3              factor       fixed                2
ID              factor       random              12
dam             factor       random              12
x2              factor       random               2
x1*x3           interaction  fixed                2
ϵ               factor       random               5
J               covariate    fixed                1

MCMC Information:

methods                 

running MCMC for RR-BLUP...100%|████████████████████████| Time: 0:00:07


<button type="button" class="btn btn-lg btn-primary">Check Results</button> 

In [22]:
keys(out2)

Base.KeyIterator for a Dict{Any,Any} with 9 entries. Keys:
  "EBV_y2"
  "Posterior mean of polygenic effects covariance matrix"
  "EBV_y1"
  "Posterior mean of marker effects"
  "Posterior mean of marker effects covariance matrix"
  "EBV_y3"
  "Posterior mean of residual variance"
  "Posterior mean of marker effects variance"
  "Posterior mean of location parameters"

In [23]:
out2["EBV_y2"]

8×2 Array{Any,2}:
 "a1"   -5.00236
 "a2"   -1.75192
 "a3"   -4.13732
 "a4"   -4.12162
 "a5"   -4.4882 
 "a6"   -4.73917
 "a10"  -4.00252
 "a12"  -5.18423

In [24]:
out2["Posterior mean of marker effects"]

3-element Array{Any,1}:
 Any["m1" -0.0197987; "m2" -0.3126; … ; "m4" -0.183057; "m5" 0.163272]   
 Any["m1" 0.095192; "m2" -0.0360789; … ; "m4" 0.142709; "m5" 0.152601]   
 Any["m1" -0.192248; "m2" 0.0923051; … ; "m4" 0.126262; "m5" -0.00278113]